# SparkSession
Objetivo: crear una Spark session, entrada a las funcionalidades de spark. 
Con base en el video y en el repositorio de Errodringer, pero con la adecuación y adaptación a mi version de pyspark y python. 

https://github.com/errodringer/CursoBigData_PySpark/tree/master
https://www.youtube.com/playlist?list=PLYFBiuYObvKB_k0bnkI41biMjD12t44GN

Nota: para iniciar el jupyter notebook como en el vídeo, yo en mi consola puse el siguiente comando porque no me lo reconoce. 

python -m notebook

In [64]:
import findspark 
findspark.init()
import pyspark.sql.functions as F
import pandas as pd
import numpy as np  

from pyspark.sql import SparkSession 
from pyspark.sql.functions import col, split
from pyspark.sql import Row
from pyspark import SparkConf
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [65]:
spark = SparkSession.builder.appName ('firstSession').getOrCreate()

Todas las configuraciones disponibles en Spark:
https://spark.apache.org/docs/latest/configuration.html

In [66]:
 spark

# DataFrame

En Python, un DataFrame es una estructura de datos bidimensional que se encuentra en la biblioteca pandas. Está diseñado para almacenar y manipular datos de manera similar a una hoja de cálculo o una tabla de base de datos relacional. 
No me dejo crear el df a partir de una lista si no importaba pandas, busque los mil errores y esta fue una solición. 

Crear Tabla
a) A partir de una lista 

En este caso se utilizo pandas aunque no es lo más recomendable porque se utilizan más recursos de la computadora, por lo que tendríamos procesamientos más lentos. 


In [67]:
columnas = ["id", "nombre", "letra"]
lista_1 = [(1, "Errodringer", "a"), (2, "Paco", "b"), (3, "Hola", "c"), (4, "Adios", "d")]

In [68]:
df_1 = pd.DataFrame(lista_1, columns= columnas)
df_1

,id,nombre,letra
0,1,Errodringer,a
1,2,Paco,b
2,3,Hola,c
3,4,Adios,d


b) A partir de un CSV o txt 

Es uno de los formatos más comunes, de manera general es df = spark.read.formato(path) aunque no son los únicos formatos ni las unicas variantes. Con el .csv me toma las primeras dos lineas como el header y en el .txt no reconoce el header. 

Para este ejemplo al leer el csv y mostrarlo con show, se observa que coloca los datos en una sóla columna, por lo que se utiliza la funcion .withColumn para reesctibir el schema. Se probaron otras formas de inferir el schema y hacer directamente un split pero ninguna funciono a que el resultado fuera, dos columnas con encabezado.Los numeros de la  función withColumn indican las posiciones que toma la columna que se crea y toma en cuenta los espacios. 


En el caso que se encuentren separados por comas tendría que buscar el comando que reconozca esta separación, por el momento me manejo sólamente por posiciones y espacios de columna. 

Coloque el .drop directamente al nombrar las columnas aunque se puede poner fuera. Simplemente corta y elimina la columna de los datos crudos. 


In [69]:
df_2 = spark.read.csv("data\\DBS_LW_ModA_highres.txt", header=True, inferSchema=True )
df_2.printSchema()


root
 |-- microns  transmission: string (nullable = true)



In [70]:
df_2 = df_2.withColumn("microns", F.substring(F.col("microns  transmission"),0,7))\
    .withColumn("transmission", F.substring(F.col("microns  transmission"),7,15))\
    .drop("microns  transmission")
df_2.printSchema()
df_2.count()
df_2.first()
df_2.show(50)
df_2.collect()
nombres_columnas = df_2.columns
num_columns = len(df_2.columns)
print("Número de columnas:", num_columns)


root
 |-- microns: string (nullable = true)
 |-- transmission: string (nullable = true)

+-------+------------+
|microns|transmission|
+-------+------------+
|2      | 1.93996E-05|
|2.001  | 1.79938E-05|
|2.002  | 1.67365E-05|
|2.003  | 1.56101E-05|
|2.004  | 1.45996E-05|
|2.005  | 1.36915E-05|
|2.006  | 1.28742E-05|
|2.007  | 1.21374E-05|
|2.008  | 1.14722E-05|
|2.009  | 1.08703E-05|
|2.01   | 1.03247E-05|
|2.011  | 9.82864E-06|
|2.012  | 9.37634E-06|
|2.013  | 8.96236E-06|
|2.014  | 8.58179E-06|
|2.015  | 8.23018E-06|
|2.016  | 7.90358E-06|
|2.017  | 7.59852E-06|
|2.018  | 7.31207E-06|
|2.019  | 7.04185E-06|
|2.02   | 6.78604E-06|
|2.021  | 6.54336E-06|
|2.022  | 6.31297E-06|
|2.023  | 6.09444E-06|
|2.024  | 5.88758E-06|
|2.025  | 5.69239E-06|
|2.026  | 5.50893E-06|
|2.027  | 5.33726E-06|
|2.028  | 5.17737E-06|
|2.029  | 5.02919E-06|
|2.03   | 4.89252E-06|
|2.031  | 4.76713E-06|
|2.032  | 4.65268E-06|
|2.033  | 4.54879E-06|
|2.034  | 4.45506E-06|
|2.035  | 4.37107E-06|
|2.036  |  4.2

# Filtros

In [80]:
df_2.filter((col('microns') > 2) & (col('transmission') .between(0.935, 0.937))
).show()

+-------+------------+
|microns|transmission|
+-------+------------+
|3.004  | 0.935388099|
|3.005  | 0.935807141|
|3.006  | 0.936212417|
|3.007  | 0.936603099|
|3.008  | 0.936978404|
|3.05   | 0.936888018|
|3.051  | 0.936645444|
|3.052  | 0.936403758|
|3.053  | 0.936163889|
|3.054  | 0.935926744|
|3.055  | 0.935693212|
|3.056  | 0.935464161|
|3.057  | 0.935240431|
|3.058  | 0.935022836|
|3.094  | 0.935184363|
|3.095  | 0.935431772|
|3.096  | 0.935689107|
|3.097  | 0.935955849|
|3.098  | 0.936231463|
|3.099  | 0.936515399|
+-------+------------+
only showing top 20 rows

